In [1]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
import csv
from pyspark.sql.functions import desc, avg, split, countDistinct, explode
from pyspark.sql import SparkSession
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import split, col

In [1]:
# pip install pyspark

     |████████████████████████████████| 281.3 MB 117 kB/s s eta 0:00:01
     |████████████████████████████████| 199 kB 44.0 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764020 sha256=dd12afc3f0100806a6ad8e6c48a735050b0cd279f6c9a5eac03de83c9d193879
  Stored in directory: /homes/mjalali/.cache/pip/wheels/05/75/73/81f84d174299abca38dd6a06a5b98b08ae25fce50ab8986fa1
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [3]:
file = '/data/dataprocessing/interproscan/all_bacilli.tsv'
sc=SparkContext('local[16]')
df = SQLContext(sc).read.csv(file, sep=r'\t', header=False, inferSchema= True)
df.show(2)



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/26 22:51:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/26 22:51:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


/homes/mjalali/.local/lib/python3.9/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/homes/mjalali/.local/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=66>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/homes/mjalali/.local/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/homes/mjalali/.local/lib/python3.9/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback 

Py4JError: An error occurred while calling o31.csv

In [2]:
spark = SparkSession.builder.master("local[16]").getOrCreate()
file = '/data/dataprocessing/interproscan/all_bacilli.tsv'
df = spark.read.csv(file, sep=r'\t', header=False, inferSchema= True)

df.show(2)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/27 14:42:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------------------+--------------------+---+-------+---------+--------------------+---+---+-------+---+----------+---------+--------------------+----+----+
|                 _c0|                 _c1|_c2|    _c3|      _c4|                 _c5|_c6|_c7|    _c8|_c9|      _c10|     _c11|                _c12|_c13|_c14|
+--------------------+--------------------+---+-------+---------+--------------------+---+---+-------+---+----------+---------+--------------------+----+----+
|gi|29898682|gb|AA...|92d1264e347e14924...|547|TIGRFAM|TIGR03882|cyclo_dehyd_2: ba...|  2|131|1.6E-21|  T|25-04-2022|IPR022291|Bacteriocin biosy...|   -|   -|
|gi|29898682|gb|AA...|92d1264e347e14924...|547|TIGRFAM|TIGR03604|TOMM_cyclo_SagD: ...|161|547|    0.0|  T|25-04-2022|IPR027624|Thiazole/oxazole-...|   -|   -|
+--------------------+--------------------+---+-------+---------+--------------------+---+---+-------+---+----------+---------+--------------------+----+----+
only showing top 2 rows



In [26]:
# distinct protein annotations
dpa = df.select(countDistinct("_c11"))
dpa.show()

+--------------------+
|count(DISTINCT _c11)|
+--------------------+
|                9704|
+--------------------+



In [32]:
# distinct protein annotations
dpa = df.select(countDistinct("_c4"))
dpa.show()
df.count()

+-------------------+
|count(DISTINCT _c4)|
+-------------------+
|              24488|
+-------------------+



4200591

In [25]:
df2 = df.groupBy("_c0").count().agg({'count':'mean'})
result = df2.take(1)[0]['avg(count)']
print(result)

11.490524386574391


In [26]:
df2 = df.filter("_c13 != '-'")
df2 = df2.select(split(col("_c13"),"\|").alias('GO'))
df2 = df2.select(explode(df2.GO))
df2 = df2.groupBy("col").count().sort(col("count"), ascending=False)
result = df2.take(1)[0]['col']
print(result)

GO:0005524


In [29]:
df2 = df.select((col("_c7") - col("_c6")).alias('Diff'))
df2 = df2.agg({'Diff' : 'mean'})
result = df2.take(1)[0]['avg(Diff)']
print(result)

131.73178916966685


In [51]:
df2 = df.filter("_c11 != '-'")
df2 = df2.groupBy("_c11").count().sort(col("count"), ascending=False)
result = []
for res in df2.take(10):
    result.append(res[0])
print(result)

['IPR027417', 'IPR002347', 'IPR003439', 'IPR036388', 'IPR036259', 'IPR003593', 'IPR036390', 'IPR036291', 'IPR000515', 'IPR001789']


In [53]:
#6
df2 = df.filter("_c11 != '-'")
df2 = df2.select("_c11",((col("_c7")-col("_c6"))/col("_c2")).alias("Pro"))
df2 = df2.filter("Pro >= 0.9").sort("Pro", ascending=False)
result = []
for res in df2.take(10):
    result.append(res[0])
print(result)

['IPR006308', 'IPR006308', 'IPR006308', 'IPR006308', 'IPR006308', 'IPR012103', 'IPR006308', 'IPR006308', 'IPR006308', 'IPR006308']


In [54]:
#7
df2 = df.filter("_c12 != '-'")
df2 = df2.select(split(col("_c12"),"\/| |-|,").alias('W'))
df2 = df2.select(explode(df2.W))
df2 = df2.filter("col != ''")
df2 = df2.groupBy("col").count().sort(col("count"), ascending=False)
result = []
for res in df2.take(10):
    result.append(res[0])
print(result)

['domain', 'like', 'superfamily', 'protein', 'binding', 'terminal', 'type', 'C', 'DNA', 'hydrolase']


In [6]:
#8
df2 = df.filter("_c12 != '-'")
df2 = df2.select(split(col("_c12"),"\/| |-|,").alias('W'))
df2 = df2.select(explode(df2.W))
df2 = df2.filter("col != ''")
df2 = df2.groupBy("col").count().sort(col("count"), ascending=True)
result = []
for res in df2.take(10):
    result.append(res[0])

+--------------------+-----+
|                 col|count|
+--------------------+-----+
|        CHLOROPLASTS|    1|
|Phycocyanobilin:f...|    1|
|             DUF2555|    1|
|             DUF4917|    1|
|               Ycf34|    1|
|                CsgG|    1|
|             sll0784|    1|
|               Curli|    1|
|           UCP020893|    1|
|               PICOT|    1|
+--------------------+-----+
only showing top 10 rows



In [22]:
#9
df2 = df.filter("_c11 != '-'")
df2 = df2.select("_c11", "_c12",((col("_c7")-col("_c6"))/col("_c2")).alias("Pro"))
df2 = df2.filter("Pro >= 0.9").sort("Pro", ascending=False)
df2 = df2.limit(10)
df2 = df2.filter("_c12 != '-'")
df2 = df2.select(split(col("_c12"),"\/| |-|,").alias('W'))
df2 = df2.select(explode(df2.W))
df2 = df2.filter("col != ''")
df2 = df2.groupBy("col").count().sort(col("count"), ascending=False)
result = []
for res in df2.take(10):
    result.append(res[0])

+----------+-----+
|       col|count|
+----------+-----+
|       III|    9|
|polymerase|    9|
|      Gram|    9|
|  positive|    9|
|      type|    9|
|     alpha|    9|
|   subunit|    9|
|       DNA|    9|
| Peptidase|    1|
|       S8A|    1|
+----------+-----+
only showing top 10 rows



In [56]:
#10
df2 = df.filter("_c11 != '-'")
df2 = df2.select("_c0", "_c11", "_c2")
df2 = df2.groupBy("_c0", "_c2").agg({"_c11":"count"})

R2 = df2.stat.corr("_c2", "count(_c11)") ** 2
output = []
result = R2
output.append([10, result])


In [60]:
with open("output.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Question Number', 'Result'])
    for res in output:
        writer.writerow(res)

TypeError: 'float' object is not iterable

In [ ]:
show, collect, avg, distictcount, agg => aggregate, select, sort, filter

In [48]:
A1 = df.filter(df.InterPro_accession!="-").select(countDistinct("InterPro_accession"))
A1.show(2)

AttributeError: 'DataFrame' object has no attribute 'InterPro_accession'

In [ ]:
A1_1 = A1.collect()[0][0]
A1_1

In [ ]:
A1_2 = A1._sc._jvm.PythonSQLUtils.explainString(A1._jdf.queryExecution(), 'simple')
A1_2